# DocTour 

In [384]:
import pandas as pd

In [385]:
class singleDoc(object):
    def __init__(self, name,doc="",level = -1):
        self.name = name.split(".")[-1]
        self.doc = doc if doc else ""
        self.parents = []
        self.kids = []
        self.names = []
        self.names.append(name)
        self.level = level
        
    def __repr__(self):
        return f"[{self.name}]"
#         return f"""
#         [Name:\t{self.name}][Level:\t{self.level}][Names:{len(self.names)}]
#         """
    
    def new_parent(self,parent):
        if parent:
            self.parents.append(parent)
            parent.kids.append(self)
            
    def new_name(self,nname):
        self.names.append(nname)
        
    def to_dicts(self,*cols):
        return dict((col,getattr(self,col)) for col in cols)


In [386]:
def mv_attr(*args):
    args_kv = dict((a,f"__{a}__") for a in args)
    def move(src,dst):
        for da,sa in args_kv.items():
            if hasattr(src, sa):
                setattr(dst, da, getattr(src, sa))
    return move

In [387]:
moves = mv_attr("doc")

In [388]:
class docTour(object):
    def __init__(self,root_obj,root_name):
        self.docs = dict()
        self.root_obj = root_obj
        self.objs = dict()
        
        self.doc_parser(self.root_obj,root_name,name_chain = root_name)
        
    def mid(self,obj):
        """
        memory address
        """
        return hex(id(obj))
    
    def to_df(self):
        df = pd.DataFrame(list(i.to_dicts("name","doc","parents","kids","names","level",) for i in self.docs.values()))
        df["p_ct"] = df.parents.apply(len)
        df["k_ct"] = df.kids.apply(len)
        return df
        
    def doc_parser(self,obj,name,level = 0,name_chain = "",parent = None):
        addr = self.mid(obj)
        if addr in self.objs:
            if type(self.objs[addr])==singleDoc:
                sd = self.objs[addr]
                sd.new_name(name)
                sd.new_parent(parent)
            return None
        else:
            self.objs[addr]=name_chain
        
        if hasattr(obj,"__doc__"):
            self.docs[name_chain] = singleDoc(name_chain,level = level,)
            moves(obj,self.docs[name_chain])
            self.docs[name_chain].new_parent(parent)
            self.objs[addr] = self.docs[name_chain]

        for attr_name in dir(obj):
            if self.mid(getattr(obj,attr_name)) in self.objs:
                continue
            elif "__" not in attr_name:
                name_chain_ = name_chain+"."+attr_name
                if level<6:
                    try:
                        self.doc_parser(getattr(obj,attr_name),attr_name,level=level+1,name_chain = name_chain_,parent = self.docs[name_chain])
                    except:
                        print(f"[ERROR]>>{name_chain_}")

In [389]:
dt = docTour(pd,"pd")

[ERROR]>>pd.Categorical._dtype.type
[ERROR]>>pd.DatetimeIndex._delegate_class._scalar_type.max.asm8.flags
[ERROR]>>pd.DatetimeIndex._na_value.asm8.flags
[ERROR]>>pd.DatetimeIndex._na_value.asm8.flat.base.T
[ERROR]>>pd.DatetimeIndex._na_value.asm8.flat.base
[ERROR]>>pd.DatetimeIndex._na_value.asm8.imag.flags
[ERROR]>>pd.DatetimeIndex._na_value.asm8.imag.flat.base
[ERROR]>>pd.DatetimeIndex._na_value.asm8.imag.imag.flags
[ERROR]>>pd.Timedelta.max.min.asm8.flags
[ERROR]>>pd.Timedelta.max.min.asm8.flat.base
[ERROR]>>pd.Timedelta.max.min.asm8.imag.flags
[ERROR]>>pd._config.config.re.enum.EnumMeta
[ERROR]>>pd._config.display.locale._collections_abc.ABCMeta
[ERROR]>>pd._hashtable.np.False_.flags
[ERROR]>>pd._hashtable.np.False_.flat.base.flags
[ERROR]>>pd._hashtable.np.True_.flags
[ERROR]>>pd._hashtable.np.emath._ln2.flags
[ERROR]>>pd._hashtable.np.emath._ln2.imag.flags


/Users/salvor/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:39: FutureWarning: Timedelta.resolution is deprecated, in a future version will behave like the standard library datetime.timedelta.resolution attribute.  Use Timedelta.resolution_string instead.
/Users/salvor/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:39: FutureWarning: `weekday_name` is deprecated and will be removed in a future version. Use `day_name` instead
/Users/salvor/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:45: FutureWarning: `weekday_name` is deprecated and will be removed in a future version. Use `day_name` instead
/Users/salvor/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:45: FutureWarning: Timedelta.resolution is deprecated, in a future version will behave like the standard library datetime.timedelta.resolution attribute.  Use Timedelta.resolution_string instead.


[ERROR]>>pd._hashtable.np.ma.core.masked
[ERROR]>>pd._hashtable.np.polynomial.Chebyshev.domain.flags
[ERROR]>>pd._hashtable.np.polynomial.Chebyshev.window.flags
[ERROR]>>pd._hashtable.np.polynomial.Hermite.domain.flags
[ERROR]>>pd._hashtable.np.polynomial.Hermite.window.flags
[ERROR]>>pd._hashtable.np.polynomial.HermiteE.domain.flags
[ERROR]>>pd._hashtable.np.polynomial.HermiteE.window.flags
[ERROR]>>pd._hashtable.np.polynomial.Laguerre.domain.flags
[ERROR]>>pd._hashtable.np.polynomial.Laguerre.window.flags
[ERROR]>>pd._hashtable.np.polynomial.Legendre.domain.flags
[ERROR]>>pd._hashtable.np.polynomial.Legendre.window.flags
[ERROR]>>pd._hashtable.np.polynomial.Polynomial.domain.flags
[ERROR]>>pd._hashtable.np.polynomial.Polynomial.window.flags
[ERROR]>>pd._hashtable.np.polynomial.chebyshev.chebdomain.flags
[ERROR]>>pd._hashtable.np.polynomial.chebyshev.chebone.flags
[ERROR]>>pd._hashtable.np.polynomial.chebyshev.chebx.flags
[ERROR]>>pd._hashtable.np.polynomial.chebyshev.chebzero.flags
[

/Users/salvor/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:39: MatplotlibDeprecationWarning: 
The msg_backend_obsolete attribute was deprecated in Matplotlib 3.0 and will be removed in 3.2.
/Users/salvor/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:45: MatplotlibDeprecationWarning: 
The msg_backend_obsolete attribute was deprecated in Matplotlib 3.0 and will be removed in 3.2.
/Users/salvor/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:39: MatplotlibDeprecationWarning: 
The msg_depr attribute was deprecated in Matplotlib 3.0 and will be removed in 3.2.
/Users/salvor/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:45: MatplotlibDeprecationWarning: 
The msg_depr attribute was deprecated in Matplotlib 3.0 and will be removed in 3.2.
/Users/salvor/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:39: MatplotlibDeprecationWarning: 
The msg_depr_ignore attribute was deprecated in Matplotlib 3.0 and will be removed in 3.2.
/Use

In [390]:
gen = iter(dt.docs.items())

In [391]:
next(gen)

('pd', [pd])

In [392]:
print(len(dt.docs))

19797


In [393]:
dt.docs["pd"]

[pd]

In [394]:
dt.docs["pd"].parents

[]

In [395]:
dt.docs["pd"].names

['pd']

In [396]:
(pd.Categorical)

pandas.core.arrays.categorical.Categorical

In [397]:
stats = dt.to_df()

In [398]:
stats.sort_values(by="p_ct",ascending=False)

,name,doc,parents,kids,names,level,p_ct,k_ct
2049,astimezone,\n Convert tz-aware Timestamp to anothe...,"[[_na_value], [DatetimeTZDtype], [Index], [Int...",[],"[pd.DatetimeIndex._na_value.astimezone, constr...",3,11,0
14357,_add_numeric_operations,\n Add numeric operations to the GroupB...,"[[DataFrameGroupBy], [Path], [Accent]]",[],[pd.core.groupby.DataFrameGroupBy._add_numeric...,4,3,0
3560,compute,provide attribute-style access to a nested dict,"[[options], [options], [options]]",[],"[pd._config.config.options.compute, io, plotting]",4,3,0
9,_concat_same_type,None,"[[Categorical], [DatetimeIndex], [_dateutil_pa...",[],"[pd.Categorical._concat_same_type, _add_arithm...",2,3,0
2253,name,"str(object='') -> str\nstr(bytes_or_buffer[, e...","[[base], [base]]",[],"[pd.DatetimeTZDtype.base.name, str]",3,2,0
...,...,...,...,...,...,...,...,...
6602,real,the real part of a complex number,[[complex]],[],[pd._hashtable.np.complex.real],4,1,0
6601,imag,the imaginary part of a complex number,[[complex]],[],[pd._hashtable.np.complex.imag],4,1,0
6600,conjugate,complex.conjugate() -> complex\n\nReturn the c...,[[complex]],[],[pd._hashtable.np.complex.conjugate],4,1,0
6599,complex,Create a complex number from a real part and a...,[[np]],"[[conjugate], [imag], [real]]",[pd._hashtable.np.complex],3,1,3


dt.docs